## Big Query 1

### Install requirements

`pip3 install google-cloud-bigquery google-cloud-bigquery-storage pyarrow tqdm ipywidgets pandas matplotlib db-dtypes pandas-gbq`

In [ ]:
project = "cs544-spring2024" 
# this name will probably be different for you

In [ ]:
from google.cloud import bigquery

In [ ]:
bq = bigquery.Client()

In [ ]:
q = bq.query(
"""
SELECT counties.geo_id, county_name, states.state_name
FROM `bigquery-public-data.geo_us_boundaries.counties` AS counties
INNER JOIN `bigquery-public-data.geo_us_boundaries.states` AS states
ON counties.state_fips_code = states.state_fips_code
WHERE states.state_name = "Wisconsin"
LIMIT 100
"""
)
q.to_dataframe()

### Structure

"project" contains "datasets" contain "tables"

#### What datasets do I have in my project?

In [ ]:
bq.list_datasets(project)

In [ ]:
for ds in bq.list_datasets(project):
    print(ds.dataset_id)

### Dataset creation

In [ ]:
ds = bigquery.Dataset(f"{project}.lec_demo")
# ds.location = "us-central1"
bq.create_dataset(ds, exists_ok=True)

### Public datasets

In [ ]:
for ds in bq.list_datasets("bigquery-public-data"):
    print(ds.dataset_id)

### List tables

In [ ]:
for t in bq.list_tables("bigquery-public-data.github_repos"):
    print(t.table_id)

### Extension access

In [ ]:
%load_ext google.cloud.bigquery

#### Run a query using `%%bigquery`

In [ ]:
%%bigquery
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5

#### Save a query result into `df` using `%%bigquery df`

In [ ]:
%%bigquery df
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5

In [ ]:
df

#### Python API

In [ ]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [ ]:
q = bq.query("""
SELECT *
FROM `bigquery-public-data.github_repos.languages`
LIMIT 5
""", job_config=no_cache)

In [ ]:
q.to_dataframe()

#### Total bytes processed and billed

In [ ]:
q.total_bytes_processed / 1024**2 # MB

In [ ]:
q.total_bytes_billed / 1024**2 # MB

#### How many times can we do this in the free tier?

In [ ]:
tb = 1024**4
tb / q.total_bytes_billed

#### How much will it per query after that in say Tokyo?

In [ ]:
price_per_tb = 7.5
q.total_bytes_billed / tb * price_per_tb

### Pricing factors

1. you pay for storage too (not just queries)
2. they have a minimum of 10 MB per query
3. they round up to the nearest 1 MB per query

In [ ]:
%%bigquery df
SELECT *
FROM `bigquery-public-data.github_repos.languages`
WHERE repo_name LIKE 'open-lambda/%'
LIMIT 5

In [ ]:
df